In [2]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from stable_gnn.fairness import Fair

# Data preprocesing

In [3]:
# Load LSAC dataset. We consider race as the sensitive attribute.

d = pd.read_csv('bar_pass_prediction.csv')
for x in ['ID', 'race1', 'race2', 'sex', 'bar', 'dnn_bar_pass_prediction', 'pass_bar', 'indxgrp2',
         'gender', 'grad', 'Dropout', 'fulltime', 'lsat', 'zfygpa', 'ugpa', 'zgpa', 'other', 'asian',
         'black', 'hisp']:
    del d[x]

def grouper_race(x):
    if x == 7:
        return 1
    else: 
        return 0

def grouper_gpa(x):
    if x>3.4:
        return 2
    elif x <3.1:
        return 0
    else:
        return 1

d['gpa'] = d['gpa'].apply(grouper_gpa)
d['race'] = d['race'].apply(grouper_race)
d = d.rename(columns = {'gpa': 'target', 'race': 'attr'})
d = pd.get_dummies(d, drop_first=True)
d = d.dropna(how='any')

# Fit & predict a decent classifier over data

In [4]:
y = d.drop(['target'], axis=1) 
x = d['target']
y_train, y_test, x_train, x_test = train_test_split(y, x)
lg = MLPClassifier()
lg.fit(y_train, x_train)
lg_preds = lg.predict(y_test)
accuracy_score(lg_preds, x_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001186 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 424
[LightGBM] [Info] Number of data points in the train set: 15320, number of used features: 23
[LightGBM] [Info] Start training from score -1.110960
[LightGBM] [Info] Start training from score -1.016623
[LightGBM] [Info] Start training from score -1.174600


0.8519678872136284

# Run FMCLP algorithm 

In [5]:
# initialize initial classifer and Fair class object.
initial_classifier = MLPClassifier(verbose=-1)
fairness = Fair(dataset=d, estimator=initial_classifier)

In [6]:
%%time

# run FMCLP algorithm
res = fairness.run(number_iterations=30,
                   interior_classifier="rf",
                   multiplier=30)

CPU times: user 17.4 s, sys: 3.91 s, total: 21.3 s
Wall time: 18.8 s


Before evaluating result we define an auxiliary function to print results. 

In [13]:
def print_results(res):
    accuracy_absolute_loss = res['accuracy_of_initial_classifier'] - res['accuracy_of_fair_classifier']
    accuracy_percentage_loss = accuracy_absolute_loss/ res['accuracy_of_initial_classifier']*100
    fairness_absolute_improvement = res['fairness_of_initial_classifier_diff'] - res['fairness_of_fair_classifier_diff'] 
    fairness_percentage_improvement = fairness_absolute_improvement/ res['fairness_of_initial_classifier_diff']*100
    
    f_accuracy = f"Accuracy of initial classifier is {res['accuracy_of_initial_classifier']:0.4f}, while accuracy of fair classifier is \
{res['accuracy_of_fair_classifier']:0.4f}. Accuracy loss is {accuracy_absolute_loss:0.4f}; it has decreased on \
{accuracy_percentage_loss:0.4f}%."

    f_fairness = f"Cuae-difference of initial classifier is {res['fairness_of_initial_classifier_diff']:0.4f}, while cuae-difference of fair \
classifier is {res['fairness_of_fair_classifier_diff']:0.4f}. Fairness improvement is {fairness_absolute_improvement:0.4f}; it has \
increased on {fairness_percentage_improvement:0.4f}%. " 

    print("")
    print(f_accuracy)
    print("")
    print(f_fairness)
    return {'fair_accuracy': res['accuracy_of_fair_classifier'],
            'initial_accuracy': res['accuracy_of_initial_classifier'],
            'fair_fairness': res['fairness_of_fair_classifier_diff'] ,
            'initial_fairness': res['fairness_of_initial_classifier_diff']}

Now we evaluate results. We see small loss in accuracy and fairness increase.

In [14]:
print_results(res)


Accuracy of initial classifier is 0.8418, while accuracy of fair classifier is 0.8275. Accuracy loss is 0.0143; it has decreased on 1.6981%.

Cuae-difference of initial classifier is 0.1953, while cuae-difference of fair classifier is 0.1593. Fairness improvement is 0.0361; it has increased on 18.4636%. 


{'fair_accuracy': 0.8274916780888976,
 'initial_accuracy': 0.8417857842177403,
 'fair_fairness': 0.15926052721670658,
 'initial_fairness': 0.1953244974812931}

# Different parameters for FMCLP algorithm.

In [19]:
results = []
for interior_classifier in ['rf', 'dt', 'knn', 'lr']:
    for multiplier in [10, 20, 30, 40]:
        res = fairness.run(number_iterations=30,
                       interior_classifier=interior_classifier,
                       multiplier=multiplier)
        results.append(res)
        print(f"Parameters: interior_classifier {interior_classifier}, multiplier {multiplier}")
        print_results(res)
        print("")
        print("")

Parameters: interior_classifier rf, multiplier 10

Accuracy of initial classifier is 0.8418, while accuracy of fair classifier is 0.8263. Accuracy loss is 0.0155; it has decreased on 1.8376%.

Cuae-difference of initial classifier is 0.1953, while cuae-difference of fair classifier is 0.1448. Fairness improvement is 0.0505; it has increased on 25.8591%. 


Parameters: interior_classifier rf, multiplier 20

Accuracy of initial classifier is 0.8418, while accuracy of fair classifier is 0.8281. Accuracy loss is 0.0137; it has decreased on 1.6283%.

Cuae-difference of initial classifier is 0.1953, while cuae-difference of fair classifier is 0.1530. Fairness improvement is 0.0423; it has increased on 21.6486%. 


Parameters: interior_classifier rf, multiplier 30

Accuracy of initial classifier is 0.8418, while accuracy of fair classifier is 0.8275. Accuracy loss is 0.0143; it has decreased on 1.6981%.

Cuae-difference of initial classifier is 0.1953, while cuae-difference of fair classifier